In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import emoji 



In [2]:
datadir = "train/"
classes = ["angry","disgusted","fearful","happy","neutral","sad","surprised"]

In [3]:
train_data =[]
def create_train_data():
    for category in classes:
        path = os.path.join(datadir, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                test_img = cv2.imread(os.path.join(path,img)) #train/angry/im001 
                new_img = cv2.resize(test_img,(224,224))
                train_data.append([new_img,class_num])
            except Exception as e:
                pass

In [4]:
create_train_data()

In [5]:
print(len(train_data))

6442


In [6]:
import random 
random.shuffle(train_data)

In [7]:
X = []
y = []
for features,label in train_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1,224,224,3) 

In [8]:
X.shape

(6442, 224, 224, 3)

In [9]:
X = X/255.0;

In [10]:
Y = np.array(y)

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [12]:
model = tf.keras.applications.MobileNetV2()

In [13]:
model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [14]:
base_input = model.layers[0].input

In [15]:
base_output = model.layers[-2].output

In [16]:
final_output = layers.Dense(128)(base_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_output)

In [17]:
new_model = keras.Model(inputs = base_input,outputs = final_output)

In [18]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
new_model.fit(X,Y, epochs=25)

In [ ]:
new_model.save('Model.h5')

C:\Users\anish\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [19]:
model1 = tf.keras.models.load_model('Model.h5')

In [20]:
frame = cv2.imread('test_image.jpg')

In [21]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
faces = faceCascade.detectMultiScale(gray,1.1,4)

In [22]:
for x,y,w,h in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if len(facess) == 0:
        print("face not detected")
    else:
        for ex,ey,ew,eh in facess:
            face_roi = roi_color[ey:ey+eh, ex:ex+ew]


In [23]:
final_image = cv2.resize(face_roi,(224,224))
final_image =np.expand_dims(final_image,axis=0)
final_image = final_image/255.0

In [24]:
prediction = model1.predict(final_image)

1/1 [==============================] - 2s 2s/step


In [25]:
np.argmax(prediction)

4

In [27]:
import cv2 
rect_bgr = (255,255,255)
img = np.zeros((500,500))
text = "Some Text"
text_width, text_height = cv2.getTextSize(text, cv2.FONT_HERSHEY_TRIPLEX, 1.5,1)[0]
text_offset_x = 10
text_offset_y = img.shape[0] - 25
box_coords = ((text_offset_x,text_offset_y),(text_offset_x+text_width+2,text_offset_y-text_height-2))
cv2.rectangle(img,box_coords[0],box_coords[1],rect_bgr,cv2.FILLED)
cv2.putText(img,text,(text_offset_x,text_offset_y),cv2.FONT_HERSHEY_TRIPLEX,1.5,(0,0,0),1)

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret,frame = cap.read()
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        facess = faceCascade.detectMultiScale(roi_gray)
        if len(facess) == 0:
            print("face not detected")
        else:
            for ex,ey,ew,eh in facess:
                face_roi = roi_color[ey:ey+eh, ex:ex+ew]
    
                final_image = cv2.resize(face_roi,(224,224))
                final_image = np.expand_dims(final_image,axis=0)
                final_image = final_image/255.0

    preds = model1.predict(final_image)

    if np.argmax(preds)==0:
        status = "Angry"
        print(emoji.emojize('Current emotion is :angry_face:'))
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
    elif np.argmax(preds)==1:
        status = "Disgusted"
        print(emoji.emojize('Current emotion is :face_with_rolling_eyes:'))
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
    elif np.argmax(preds)==2:
        status = "Fearful"
        print(emoji.emojize('Current emotion is :fearful_face:'))
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
    elif np.argmax(preds)==3:
        status = "happy"
        print(emoji.emojize('Current emotion is :smiling_face_with_smiling_eyes:'))
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
    elif np.argmax(preds)==4:
        status = "Neutral"
        print(emoji.emojize('Current emotion is :neutral_face:'))
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
    elif np.argmax(preds)==5:
        status = "Sad"
        print(emoji.emojize('Current emotion is :sad_but_relieved_face:'))
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
    elif np.argmax(preds)==6:
        status = "Surprised"
        print(emoji.emojize('Current emotion is :flushed_face:'))
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))
    else:
        status = "None"
        print('face is not visible')
        x1,y1,w1,h1 = 0,0,175,75
        cv2.rectangle(frame,(x1,x1),(x1+w1,y1+h1),(0,0,0),-1)
        cv2.putText(frame,status,(x1+int(w1/10),y1+int(h1/2)),cv2.FONT_HERSHEY_TRIPLEX,0.7,(0,0,255),2)
        cv2.putText(frame,status,(100,150),cv2.FONT_HERSHEY_TRIPLEX,3,(0,0,255),2,cv2.LINE_4)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255))

    cv2.imshow("Face det",frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 104ms/step
Current emotion is 😐
1/1 [==============================] - 0s 85ms/step
Current emotion is 😥
1/1 [==============================] - 0s 66ms/step
Current emotion is 😐
1/1 [==============================] - 0s 64ms/step
Current emotion is 😥
1/1 [==============================] - 0s 62ms/step
Current emotion is 😐
1/1 [==============================] - 0s 65ms/step
Current emotion is 😥
1/1 [==============================] - 0s 65ms/step
Current emotion is 😐
1/1 [==============================] - 0s 64ms/step
Current emotion is 😐
1/1 [==============================] - 0s 63ms/step
Current emotion is 😐
1/1 [==============================] - 0s 73ms/step
Current emotion is 😥
1/1 [==============================] - 0s 65ms/step
Current emotion is 😥
1/1 [==============================] - 0s 66ms/step
Current emotion is 😐
1/1 [==============================] - 0s 64ms/step
Current emotion is 😐
1/1 [==============================] - 0s 69ms/ste

KeyboardInterrupt: 

["angry","disgusted","fearful","happy","neutral","sad","surprised"]